In [ ]:
import os
import sys
sys.path.append('..')
sys.path.append('../Automatic-Circuit-Discovery/')
sys.path.append('../tracr/')
import IPython
ipython = get_ipython()
if ipython is not None:
    ipython.magic('load_ext autoreload')
    ipython.magic('autoreload 2')
import torch as t
from torch import Tensor

from acdc.docstring.utils import get_all_docstring_things
device = t.device("cuda" if t.cuda.is_available() else "CPU")
print(device)

In [ ]:
all_docstring_items = get_all_docstring_things(num_examples=40, seq_len=5, device=device, metric_name='docstring_metric', correct_incorrect_wandb=False)

tl_model = all_docstring_items.tl_model
validation_metric = all_docstring_items.validation_metric
validation_data = all_docstring_items.validation_data
validation_labels = all_docstring_items.validation_labels
validation_patch_data = all_docstring_items.validation_patch_data
test_metrics = all_docstring_items.test_metrics
test_data = all_docstring_items.test_data
test_labels = all_docstring_items.test_labels
test_patch_data = all_docstring_items.test_patch_data

In [ ]:
def neg_abs_docstring_metric(logits):
    return -abs(test_metrics['docstring_metric'](logits))

In [ ]:
from ACDCPPExperiment import ACDCPPExperiment
import numpy as np
THRESHOLDS = [-10]#np.logspace(-4, 1, num=20, base=5) # [0.0473844]
# I'm just using one threshold so I can move fast!

tl_model.reset_hooks()
RUN_NAME = 'acdc_edges'
acdcpp_exp = ACDCPPExperiment(
    model=tl_model,
    clean_data=test_data,
    corr_data=test_patch_data,
    acdc_metric=neg_abs_docstring_metric,
    acdcpp_metric=test_metrics['docstring_metric'],
    thresholds=THRESHOLDS,
    run_name=RUN_NAME,
    verbose=True,
    attr_absolute_val=True,
    save_graphs_after=0,
    run_acdcpp=True,
    run_acdc=False,
    pruning_mode='edge',
    no_pruned_nodes_attr=1,
)

pruned_heads, num_passes, acdcpp_pruned_attrs, acdc_pruned_attrs, edges_after_acdcpp, edges_after_acdc = acdcpp_exp.run()

## Just run the cell below to save ACDC++ attribution scores

In [ ]:
print(RUN_NAME)

import json
with open(f'{RUN_NAME}_acdcpp_scores.json', 'w') as f:
    json.dump(acdcpp_pruned_attrs, f)

In [13]:
max(acdcpp_pruned_attrs[-10].values()) # 4.55, for blocks.1.hook_k_input[:, :, 4]blocks.0.hook_resid_pre[:]

4.554133415222168